Será que podemos comparar ideologias políticas a partir de tweets?

ideologia: conjunto de convicções filosóficas, sociais, políticas etc. de um indivíduo ou grupo de indivíduos.

Comparações:
    - análise gráfica comparativa da frequência de palavras ou relacionadas a tópicos: 
        religião, educação, orientação sexual, identidade de gênero, economia, saúde, porte de armas, combate a pobreza
            ex: Quantidade de tweets do Bolsonaro falando sobre drogas é muito maior que a do Lula, que mostra maior favor a política anti-drogas
            (Buscar se a quantidade de palavras segue a mesma tendência)

    - Comparação percentual entre palavras específicas em relação ao total

    -! análise comparativa de tweets antes e depois de eleições

Observações:
    - contam retweets? -> Usar a função clean() para filtrar

Proximos passos:
Otimizar função de limpeza -> lemanização e estemização para remoção de radicais
Comparar temas por datas específicas

In [31]:
# Imports section
import json
import nltk
import re
import pandas as pd
import unicodedata as uni

In [22]:
def get_data():
    '''Open and read dataset files, saving contents as raw data.'''
    with open('dataset/jairbolsonaro.json','r') as file_01:
        data_01=json.load(file_01)
    with open('dataset/LulaOficial.json','r') as file_02:
        data_02=json.load(file_02)
        
    return data_01,data_02
# get_data()[0][0]

In [23]:
def extract_tweets():
    '''Store tweets in lists without metadata'''
    data_01,data_02 = get_data()
    bolso_tweets = []
    for el in data_01:
        bolso_tweets.append(el['full_text'])

    lula_tweets = []
    for el in data_02:
        lula_tweets.append(el['full_text'])
        
    return bolso_tweets, lula_tweets

In [24]:
def compare(bolso_tweets,lula_tweets,subject):
    # Fix_01:Currently isn't useful because it is too restricted, doesn't account for synonyms and related words
     
    '''Compare tweets from both sides based on a subject and return a dataframe for comparison'''
    # Look into Bolsonaro's tweets for the subject and store tweets in a list
    b_selected = []
    for tweet in bolso_tweets:
        if subject.lower() in tweet.lower():
            b_selected.append(tweet)

    # Look into Lula's tweets for the subject and store tweets in a list
    l_selected = []
    for tweet in lula_tweets:
        if subject.lower() in tweet.lower():
            l_selected.append(tweet)

    # Create two series based on the selected tweets lists
    b = pd.Series(b_selected,name='Bolsonaro')
    l = pd.Series(l_selected,name='Lula')

    # Create dataframe concatenating the two series
    pd.set_option('display.max_colwidth', None)
    df=pd.concat([b,l],axis=1)
    
    return df

In [66]:
def clean(tweet):
    '''Normalizes text, removes links, hashtags, mentions, pontuation, emogis and removes stopwords from a single tweet and returns a list with the remaining words'''
    
    stopwords=['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam']
    # normalize text
    tweet = uni.normalize('NFD', tweet).encode('ASCII', 'ignore').decode('utf-8')
    # remove links
    url_pattern=r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'\".,<>?«»“”‘’]))'
    # remove hashtags
    tags_pattern=r'\#([a-zA-Z0-9_]{1,50})'
    # remove mentions
    mentions_pattern=r'\@([a-zA-Z0-9_]{1,50})'    
    # remove pontuation and emogis 
    rest_pattern=r'[^\w\s]'
    # compile all patterns
    patterns = [url_pattern,tags_pattern,mentions_pattern,rest_pattern]
    for pattern in patterns:
        tweet = re.sub(pattern,'',tweet)
    
    clean_tweet=[word for word in tweet.split() if word not in stopwords]
    
    return clean_tweet


In [67]:
bolso_tweets, lula_tweets = extract_tweets()
a=clean(bolso_tweets[3])
print(a)


['Trecho', 'entrevista', 'vivo', 'Jornal', 'Nacional', '08102018']


In [28]:
def count_words(tweets,min=0):
    '''Count the ocurrence of key words in the tweets'''
    words={}
    selected_words=[]
    for tweet in tweets:
        for word in clean(tweet.split()):
            if word in words.keys():
                words[word] += 1
            else:
                words[word] = 1

                
    # Filter the words into a list based on a minimum number of ocurrences
    for word, ocurrence in words.items():
        if ocurrence > min:
            selected_words.append(word)

    # return selected_words
    return sorted(words,key=words.get,reverse=True)

In [29]:
bolso_tweets, lula_tweets = extract_tweets()
# count_words(bolso_tweets)
print(bolso_tweets,lula_tweets)

['EM "DITADURA" SEM PAREDÃO, ATÉ CHICO ALENCAR É VALENTÃO. Mostrem ao povo as marcas da tortura. Vcs estão d bolso cheio do Bolsa-Ditadura.', 'Bom dia! 🇧🇷 #tbt com o amigo "Canguru", que já deixou este plano. Um forte abraço a todos! https://t.co/fNPQUTBnfi', '- Para descontrair. Proibido queimar ovo. (Kkkk) https://t.co/dmqM8DT8Fz', 'Trecho de entrevista ao vivo para o Jornal Nacional (08/10/2018): https://t.co/GFRZznKrZS', 'Querem criar o fundão bilionário na Reforma Política mas alegam não ter dinheiro para aplicabilidade plena do voto impresso! Contem outra!', 'Por @Rconstantino : O maior investimento financeiro do governo Petista foi em... Cuba. E c/ muito sigilo. http://t.co/HHHaCOKd7r', '@guilhermesousa Kkkkkkk... tá ok, roteirista da tv globo.', '@Gus_Vilarinho - 👍🇧🇷', 'A LEGALIZAÇÃO DO LATROCÍNIO. Via @CanalHipocritas https://t.co/NQGuqBwe4U', 'Prefeitura do Rio/RJ-Lançamento pré-candidatura de @FlavioBolsonaro - (30/05), auditório da Bolsa de Valores (18h). https://t.co/NfVNJ